#  Pandemic Mortality Trends: A Deep Dive into Provisional COVID-19 Death Data

## Introduction

This dataset captures COVID-19 data across different regions, focusing on mortality rates and trends. The data includes key metrics such as COVID-related deaths, percentage of total deaths due to COVID, weekly percentage changes, and crude COVID rates. The data spans a defined period with start and end dates, and is categorized by residence jurisdiction, with the "total" group aggregating data for various time periods.

In this project, I will analyze COVID-19 data to address the following questions:

1. Which jurisdiction had the highest number of COVID-19 deaths for the latest data period end date?
1. What is the week-over-week percentage change in the crude COVID rate for all jurisdictions and groups, and how do they rank.Retrieve the top 10 results?
1. Which are the top 5 jurisdictions with the highest percentage difference between aa_COVID_rate and the overall crude COVID rate for the latest data period end date?
1. What is the average number of COVID deaths per week for each jurisdiction residence and group, based on the latest four data period end dates?
1. Which jurisdictions had non-zero COVID deaths and no missing data for the latest data period end date?
1. How has the percentage of total deaths attributed to COVID changed on a week-over-week basis for the top 10 jurisdictions with the highest total COVID deaths, starting from March 1, 2020?
1. What are the cumulative COVID deaths for each jurisdiction up to the latest data period end date?
1. Which Top 10 jurisdiction experienced the highest percentage increase in COVID deaths from the previous week, and what were the actual death numbers for each week?






## Information on the Dataset

The key columns of the dataset are:
* **Date_of_Datarecorded:** This column captures the exact date when the data was recorded.
* **Residence_Jurisdiction:** Represents the region or locality for which the data is collected. It helps in analyzing trends and metrics specific to different geographical areas.
* **Period:** This column indicates the time period (e.g., weekly, total) for which the data is aggregated. It allows for analysis based on different periods of data collection.
* **Data_Period_start:** Denotes the start date of the specific data period. It helps in determining the exact duration of the reported figures.
* **Data_Period_end:** This is the end date of the specific data period. Together with the start date, it defines the window during which the data was collected.
* **Covid_Deaths:** The number of deaths attributed to COVID-19 in the specified jurisdiction during the data period. This is a key metric for analyzing the severity of the pandemic in different regions.
* **Total_Covid_Prct:** The percentage of total deaths that were caused by COVID-19. This percentage highlights the proportion of deaths directly related to the pandemic within each jurisdiction.
* **Pct_Change_Weekly:** The week-over-week percentage change in COVID-related metrics, such as deaths or infection rates. It is crucial for identifying spikes or declines in the spread of COVID-19 over time.
* **Pct_Diff_Weekly:** Represents the difference between percentage changes over consecutive weeks. This allows for analyzing the volatility or consistency of COVID-related trends.
* **Crude_Covid_Rate:** This column provides the crude rate of COVID-19 deaths per a certain population size (e.g., per 100,000 people). It is used to compare the relative impact of COVID-19 across regions, accounting for population differences.
* **aa_Covid_Rate:** The age-adjusted COVID-19 death rate, which factors in the age distribution of the population. This is important for making more accurate comparisons across regions with varying age demographics.

## Data Exploration
Let's have look at the data;

In [ ]:
SELECT column_name ,data_type
FROM INFORMATION_SCHEMA.columns

|column_name|data_type|
|-----------|---------|
|Date_of_Datarecorded|date|
|Residence_Jurisdiction|	varchar|
|Period	|varchar|
|Data_Period_start|	date|
|Data_Period_end|	date|
|Covid_Deaths|	int|
|Total_Covid_Prct|	float|
|Pct_Change_Weekly|	float|
|Pct_Diff_Weekly|	float|
|Crude_Covid_Rate|	float|
|aa_Covid_Rate|	float|

In [ ]:
SELECT COUNT(*) AS Row_count
FROM CovidP

There are 21952 responses.

Checking for the correct category in Group column for further analysis.

In [ ]:
#For 'TOTAL' Category
SELECT 
    SUM(Covid_Deaths) as Total_deaths, 
    min(Data_Period_start) as Start_Date,
    MAX(Data_Period_end) as End_Date
FROM 
    CovidP
WHERE
    Period = 'Total'
    
#For 'Weekly' Category
SELECT 
    SUM(Covid_Deaths) as Total_deaths, 
    min(Data_Period_start) as Start_Date,
    MAX(Data_Period_end) as End_Date
FROM 
    CovidP
WHERE
    Period = 'Weekly'
    
#For '3 month period' Category
SELECT 
    SUM(Covid_Deaths) as Total_deaths, 
    min(Data_Period_start) as Start_Date,
    MAX(Data_Period_end) as End_Date
FROM 
    CovidP
WHERE
    Period = '3 month period'

I've chosen weekly data for this analysis as it provides a properly distributed and consistent view of trends. 
In contrast, the total data represents a cumulative sum of daily figures, which can inflate the analysis  and 
the 3-month period covers only a 4-month range, making it less reliable for consistent comparison

## Data Cleaning

Checking for NULL values in each Column.

In [ ]:
SELECT 
    COUNT(*) AS Total_Records, 
    SUM(CASE WHEN Covid_deaths IS NULL THEN 1 ELSE 0 END) AS Null_Count
FROM CovidP;

SELECT 
    COUNT(*) AS Total_Records, 
    SUM(CASE WHEN Pct_diff_weekly IS NULL THEN 1 ELSE 0 END) AS Null_Count
FROM CovidP;

SELECT 
    COUNT(*) AS Total_Records, 
    SUM(CASE WHEN Pct_Change_Weekly IS NULL THEN 1 ELSE 0 END) AS Null_Count
FROM CovidP;

SELECT 
    COUNT(*) AS Total_Records, 
    SUM(CASE WHEN Crude_Covid_Rate IS NULL THEN 1 ELSE 0 END) AS Null_Count
FROM CovidP;

**Output:**
* There are 1,512 NULL counts in Covid_deaths & Total_Covid_prct column.
* There are 12,728 NULL counts in Pct_diff_weekly column.
* There are 13,299 NULL counts in Pct_Change_Weekly.
* There are 2490 NULL counts in Crude_covid_rate.

Treating the NULL values with 0 and mean according to their significance. 

In [ ]:
# Replacing the NULL values in Covid_deaths , Total_Covid_prct and Crude_covid_rate with '0'

UPDATE CovidP SET Covid_Deaths=0 WHERE Covid_Deaths IS NULL;
UPDATE CovidP SET Total_Covid_prct=0 WHERE Total_Covid_prct IS NULL;
UPDATE CovidP SET Crude_covid_rate=0 WHERE Crude_covid_rate IS NULL;

# Replacing the NULL values in Pct_Change_Weekly and Pct_diff_weekly column using mean

UPDATE CovidP SET Pct_Change_Weekly = (
                                    SELECT ROUND(AVG(Pct_Change_Weekly),2)
                                    FROM CovidP
                                    WHERE Pct_Change_Weekly IS NOT NULL)
                                    WHERE Pct_Change_Weekly IS NULL;


UPDATE CovidP SET Pct_Diff_Weekly = (
                                    SELECT ROUND(AVG(Pct_Diff_Weekly),2)
                                    FROM CovidP
                                    WHERE Pct_Diff_Weekly IS NOT NULL)
                                    WHERE Pct_Diff_Weekly IS NULL;


## Data Analysis

**Question 1:
Which jurisdiction had the highest number of COVID-19 deaths for the latest data period end date?**

This query identifies the jurisdiction that recorded the highest number of COVID-19 deaths during the most recent weekly reporting period.

In [ ]:
SELECT TOP 1 Residence_Jurisdiction, COVID_deaths, data_Period_end
FROM CovidP
WHERE data_Period_end= ( SELECT MAX(data_Period_end) 
                         FROM CovidP
) AND Period ='weekly'
ORDER BY COVID_deaths DESC;

**Query Logic:**
* The subquery finds the latest data period end date.
* The main query then retrieves the jurisdiction with the highest number of COVID-19 deaths during that period, filtering for weekly data and ordering the results in descending order of deaths.


**Answer:**
For the latest weekly data period, the jurisdiction with the highest number of COVID-19 deaths is **United States**, with **429** deaths as of **2023-04-08**.






**Question 2: What is the week-over-week percentage change in the crude COVID rate for all jurisdictions and groups, and how do they rank.Retrieve the top 10 results?**



In [ ]:
WITH Week_Over_Week AS (
    SELECT 
        Residence_Jurisdiction,
        Period,
        data_period_start,
        Data_Period_end,
        crude_COVID_rate,
        LAG(crude_COVID_rate) OVER (PARTITION BY Residence_Jurisdiction, Period ORDER BY data_period_end) AS prev_crude_COVID_rate
    FROM 
        CovidP
    WHERE 
        Period = 'weekly'
),
Pct_change_calc AS (
    SELECT
        Residence_Jurisdiction,
        Period,
        data_period_start,
        data_period_end,
        crude_COVID_rate,
        prev_crude_COVID_rate,
        CASE 
            WHEN prev_crude_COVID_rate = 0 OR prev_crude_COVID_rate IS NULL THEN NULL
            ELSE ROUND(((crude_COVID_rate - prev_crude_COVID_rate) / prev_crude_COVID_rate) * 100,2)
        END AS pct_change_wk
    FROM
        Week_Over_Week
    )
SELECT TOP 10 *
FROM 
    pct_change_calc
WHERE 
    pct_change_wk IS NOT NULL 
ORDER BY
    pct_change_wk DESC

**Query Logic:**
* The first Common Table Expression (CTE) named Week_Over_Week retrieves the crude COVID rates along with the previous week's rates for each jurisdiction and period, using the LAG() function. This allows for comparison of the current week's rate with that of the previous week.
* The second CTE, Pct_change_calc, calculates the percentage change in crude COVID rates. If the previous week's rate is zero or null, the result will also be null; otherwise, it computes the percentage change using the formula:((current - previous)/previous)*100.
* The final query selects the top 10 jurisdictions with the highest percentage change in crude COVID rates, filtering out null values and ordering the results in descending order of the percentage change.


**Output:**

|Residence_Jurisdiction|Period|data_period_start|data_period_end|crude_COVID_rate|prev_crude_COVID_rate|pct_change_wk|
|-----|-------|-----|-----|----|-----|-----|
|Michigan|	weekly|	2020-03-22|	2020-03-28|	2.3|	0.2|	1050|
|New Jersey|	weekly|	2020-03-22|	2020-03-28|	3.2|	0.4|	700|
|Region 5|	weekly|	2020-03-22|	2020-03-28|	0.8|	0.1|	700|
|New York City|	weekly|	2020-03-22|	2020-03-28|	11.8|	1.5|	686.67|
|Region 2|	weekly|	2020-03-22|	2020-03-28|	4.9	|0.7|	600|
|New York|	weekly|	2020-03-22|	2020-03-28|	2.6|	0.4|	550|
|Region 6|	weekly|	2020-03-22|	2020-03-28|	0.6|	0.1|	500|
|Louisiana|	weekly|	2020-03-22|	2020-03-28|	4.2|	0.8|	425|
|United States|	weekly|	2020-03-22	|2020-03-28|	1|	0.2	|400|
|Massachusetts|	weekly|	2020-03-29|	2020-04-04|	3.3|	0.7|	371.43|

**Question 3: Which are the top 5 jurisdictions with the highest percentage difference between aa_COVID_rate and the overall crude COVID rate for the latest data period end date?**

In [ ]:

SELECT TOP 5
    Residence_Jurisdiction, 
    aa_COVID_rate, 
    crude_COVID_rate,
    ((aa_COVID_rate - crude_COVID_rate) / NULLIF(crude_COVID_rate, 0)) * 100 AS pct_diff_aa_vs_crude
FROM 
    CovidP
WHERE 
    data_period_end = (SELECT MAX(data_period_end) FROM CovidP)
    AND crude_COVID_rate <> 0 
    AND Period = 'Weekly'
ORDER BY 
    pct_diff_aa_vs_crude DESC;  


**Query Logic:**

* The subquery retrieves the latest available data period end date from the dataset.
* The main query then selects the top 5 jurisdictions based on the percentage difference between the age-adjusted COVID rate (aa_COVID_rate) and the crude COVID rate (crude_COVID_rate).
* The percentage difference is calculated using the formula:((aa_COVID_rate - crude_COVID_rate) / crude_COVID_rate) * 100
* The NULLIF function ensures that divisions by zero in the crude_COVID_rate column are handled gracefully, avoiding errors.
* The results are filtered to include only "Weekly" data and exclude records where the crude COVID rate is zero.
* The query orders the jurisdictions by the highest percentage difference in descending order, providing insights into where the age-adjusted rate significantly deviates from the crude rate.

Output:

|Residence_Jurisdiction|	aa_COVID_rate|	crude_COVID_rate|	pct_diff_aa_vs_crude|
|---------|---------|---------|-----------|
|Region 2|	0.2|	0.2|	0|
|Region 5|	0.2|	0.2|	0|
|Michigan|	0.2|	0.2|	0|
|Minnesota|	0.285714286|	0.4|	-28.5714285|
|New York|	0.285714286|	0.4|	-28.5714285|

**Question 4: What is the average number of COVID deaths per week for each jurisdiction residence and group, based on the latest four data period end dates?**

In [ ]:
WITH LatestPeriods AS (
 
    SELECT TOP 4 Data_Period_end
    FROM CovidP 
    GROUP BY Data_Period_end 
    ORDER BY Data_Period_End DESC
)
SELECT 
    Residence_Jurisdiction,
    Period,
    Avg(Covid_deaths) as Avg_Covid_Deaths_Per_Week
FROM 
    CovidP
WHERE 
    Data_Period_End IN (SELECT Data_Period_End FROM LatestPeriods) AND Period='Weekly'
GROUP BY 
    Residence_Jurisdiction,Period
ORDER BY 
    Avg_Covid_Deaths_Per_Week DESC;

**Query Logic:**

* The subquery (LatestPeriods) identifies the most recent four Data_Period_end dates, grouping and sorting the data to get the latest periods.
* The main query calculates the average number of COVID-19 deaths per week (Avg_Covid_Deaths_Per_Week) for each Residence_Jurisdiction and Period where the period is labeled as "Weekly."
* It filters the data to include only the records that match the latest data periods obtained from the subquery.
* Finally, the results are grouped by Residence_Jurisdiction and Period and are ordered in descending order based on the calculated average weekly deaths.


**Output:**

|Residence_Jurisdiction|	Period|	Avg_Covid_Deaths_Per_Week|
|--------|--------|--------|
|United States|	weekly|	1084|
|Region 4|	weekly|	224|
|Region 5|	weekly|	223|
|Region 3|	weekly|	128|
|Region 2|	weekly|	116|
|Region 9|	weekly|	108|
|Region 6|	weekly|	97|
|California|	weekly|	73|
|Florida|	weekly|	66|
|Pennsylvania|	weekly|	64|
|Region 7|	weekly|	61|
|New York|	weekly|	61|
|Texas|	weekly|	59|
|Region 1|	weekly|	53|
|Region 10|	weekly|	49|
|Illinois|	weekly|	49|
|Michigan|	weekly|	48|
|Ohio|	weekly|	46|
|North Carolina|	weekly|	36|
|Tennessee|	weekly|	32|
|New Jersey|	weekly|	30|
|Minnesota|	weekly|	30|
|Washington|	weekly|	27|
Wisconsin| weekly| 27|
|Region 8|	weekly|	27|
|Missouri|	weekly|	24|
|Massachusetts|	weekly|	22|
|Indiana|	weekly|	22|
|Georgia|	weekly|	22|
|Arizona|	weekly|	21|
|Maryland|	weekly|	21|
|Virginia|	weekly|	20|
|Oklahoma|	weekly|	18|
|Kentucky|	weekly|	18|
|Colorado|	weekly|	15|
|South Carolina|	weekly|	15|
|West Virginia|	weekly|	14|
|Iowa|	weekly|	14|
|New York City|	weekly|	14|
|Alabama|	weekly|	13|
|Oregon|	weekly|	12|
|Mississippi|	weekly|	10|
|Connecticut|	weekly|	9|
|Kansas|	weekly|	9|
|Maine|	weekly|	8|
|Idaho|	weekly|	6|
|New Mexico|	weekly|	5|
|Nebraska|	weekly|	5|
|Nevada|	weekly|	3|
|Arkansas|	weekly|	3|
|Puerto Rico|	weekly|	3|
|Louisiana|	weekly|	2|
|Hawaii|	weekly|	0|
|Alaska|	weekly|	0|
|Delaware|	weekly|	0|
|District of Columbia|	weekly|	0|
|New Hampshire|	weekly|	0|
|North Dakota|	weekly|	0|
|Montana|	weekly|	0|
|South Dakota|	weekly|	0|
|Rhode Island|	weekly|	0|
|Wyoming|	weekly|	0|
|Utah|	weekly|	0|
|Vermont|	weekly|	0|

**Question 5: Which jurisdictions had non-zero COVID deaths and no missing data for the latest data period end date?**

In [ ]:
SELECT *
FROM covidp
WHERE Data_Period_end = (
    SELECT MAX(Data_Period_end) 
    FROM covidp
)
AND Covid_Deaths > 0
AND Residence_Jurisdiction IS NOT NULL
AND Date_of_Datarecorded IS NOT NULL
AND Period IS NOT NULL
AND Data_Period_start IS NOT NULL
AND Total_Covid_Prct IS NOT NULL
AND Pct_Change_Weekly IS NOT NULL
AND Pct_Diff_Weekly IS NOT NULL
AND Crude_Covid_Rate IS NOT NULL
AND aa_Covid_Rate IS NOT NULL
AND Period ='Weekly';

**Query Logic:**

* The subquery finds the latest data period end date by retrieving the maximum value from the Data_Period_end column.
* The main query uses the result of the subquery to filter the data for the most recent period.
* Only records with non-null values in critical columns such as Residence_Jurisdiction, Covid_Deaths, Date_of_Datarecorded, and various percentage and rate metrics are included.
* It further excludes jurisdictions with zero COVID-19 deaths by applying Covid_Deaths > 0 and focuses on the 'Weekly' period data by using Period = 'Weekly'.

**Output:**

|Date_of_Datarecorded|	Residence_Jurisdiction|	Period|	Data_Period_start|	Data_Period_end|	Covid_Deaths|	Total_Covid_Prct|Pct_Change_Weekly|Pct_Diff_Weekly|Crude_Covid_Rate|aa_Covid_Rate|
|-------|--------|-----|-----|-------|------|-----|-------|-------|--------|------|
|2023-04-12|	Region 1|	weekly|	2023-04-02|	2023-04-08|	18|	1.9|	-13.6|	-0.3|	0|	0|
|2023-04-12|	Region 2|	weekly|	2023-04-02|	2023-04-08|	70|	2.2|	-8.3|	-0.2|	0.2|	0.2|
|2023-04-12|	Region 3|	weekly|	2023-04-02|	2023-04-08|	54|	2.2|	-18.5|	-0.5|	0.2|	0|
|2023-04-12|	Region 4|	weekly|	2023-04-02|	2023-04-08|	90|	2.5|	13.6|	0.3|	0.1|	0|
|2023-04-12|	Region 5|	weekly|	2023-04-02|	2023-04-08|	109|	3.2|	10.3|	0.3|	0.2|	0.2|
|2023-04-12|	Region 6|	weekly|	2023-04-02|	2023-04-08	|30	|2.2|4.8|	0.1|	0.1|	0|
|2023-04-12|	Region 7|	weekly|	2023-04-02|	2023-04-08|	25|	2.9|	45|	0.9|	0.2|	0|
|2023-04-12|	Region 9|	weekly|	2023-04-02|	2023-04-08|	13|	1.4|	-39.1|	-0.9|	0|	0|
|2023-04-12|	Region 10|	weekly|	2023-04-02|	2023-04-08|	17|	2.3|	-28.1|	-0.9|	0|	0|
|2023-04-12|	Florida|	weekly|	2023-04-02|	2023-04-08|	21|	1.5|	-28.6|	-0.6|	0.1|	0|
|2023-04-12|	Illinois|	weekly|	2023-04-02|	2023-04-08|	35|	3.1	|24|	0.6|	0.3	|0.2|
|2023-04-12|	Maryland|	weekly|	2023-04-02|	2023-04-08|	14	|2.9|	-14.7|	-0.5|	0|	0|
|2023-04-12|	Massachusetts|weekly|	2023-04-02|	2023-04-08|	11|	2.6|	44.4|	0.8|	0|	0|
|2023-04-12|	Michigan|	weekly|	2023-04-02|	2023-04-08|	24|	3.1|	0|	0|	0.2|	0.2|
|2023-04-12|	Minnesota|	weekly|	2023-04-02|	2023-04-08|20	|4.4|	25.7|	0.9|	0.4|	0.285714286|
|2023-04-12|	Missouri|	weekly	|2023-04-02|	2023-04-08|	10|	2.7|	28.6|	0.6|	0|	0|
|2023-04-12|	New Jersey|	weekly|	2023-04-02|	2023-04-08|	14|	1.9|	35.7|	0.5|	0|	0|
|2023-04-12|	New York|	weekly|	2023-04-02|	2023-04-08|	41|	2.7|	-18.2|	-0.6|	0.4|	0.285714286|
|2023-04-12|	North Carolina|	weekly|	2023-04-02|	2023-04-08|	16|	4|	90.5|	1.9	|0|	0|
|2023-04-12|	Ohio	|weekly|	2023-04-02	|2023-04-08|	17|	3.1|	-13.9|	-0.5|	0|	0|
|2023-04-12|	Pennsylvania|	weekly|	2023-04-02|	2023-04-08|	27|	2.1|	-16|	-0.4|	0.2|	0|
|2023-04-12|	Tennessee|	weekly|	2023-04-02|	2023-04-08|	17|	3.3|	37.5|	0.9	|0|	0|
|2023-04-12|	Texas|	weekly|	2023-04-02|	2023-04-08|	22|	2.6|	44.4|	0.8	|0.1	|0|
|2023-04-12	|United States	|weekly|	2023-04-02	|2023-04-08|	429|	2.5|	0|	0|0.1|	0|
|2023-04-12|	Washington	|weekly	|2023-04-02|	2023-04-08|	15|	3.4|	13.3|	0.4|	0|	0|
|2023-04-12|	Wisconsin	|weekly|	2023-04-02|	2023-04-08|	11|3.2|	14.3|	0.4|	0|	0|

**Question 6: How has the percentage of total deaths attributed to COVID changed on a week-over-week basis for the top 10 jurisdictions with the highest total COVID deaths, starting from March 1, 2020?**

In [ ]:
WITH week_over_week AS ( 
    SELECT
        Residence_jurisdiction,
        Period,
        Data_Period_start,
        Data_Period_end,
        Total_Covid_Prct,
        LAG(Total_Covid_Prct) OVER (PARTITION BY Residence_Jurisdiction, Period ORDER BY Data_Period_Start) AS Prev_Week_Prct
    FROM 
        CovidP
    WHERE 
        Data_Period_start >= '2020-03-01'
        AND Period='Weekly'),
Pct_diff_calc AS (
    SELECT 
        Residence_jurisdiction,
        Period,
        Data_Period_start,
        Data_Period_end,
        Total_Covid_Prct,
        Prev_Week_Prct,
        CASE
            WHEN Prev_week_Prct =0 or Prev_Week_Prct is NULL THEN NULL
            ELSE ((Total_Covid_Prct-Prev_Week_Prct)/Prev_Week_Prct)*100
        END AS Pct_change_weekly  
    FROM 
        week_over_week
    WHERE 
        Period='Weekly')
SELECT top 10
    Residence_jurisdiction,
    Period,
    Data_Period_start,
    Data_Period_end,
    Total_Covid_Prct,
    Prev_Week_Prct,
    Pct_change_weekly
FROM 
    Pct_diff_calc
WHERE 
    Pct_change_weekly IS NOT NULL
ORDER BY 
    Pct_change_weekly DESC;

**Query Logic:**
* The first common table expression (CTE) week_over_week: Partitions the data by Residence_Jurisdiction and Period (for weekly periods).Uses the LAG() function to retrieve the percentage of total deaths from the previous week (Prev_Week_Prct), based on the Data_Period_start.Filters the data for weekly periods starting from March 1, 2020.
* The second CTE Pct_diff_calc:Calculates the week-over-week percentage change (Pct_change_weekly) using the current week's percentage of total COVID deaths (Total_Covid_Prct) and the previous week's percentage (Prev_Week_Prct).If the previous week's percentage is 0 or null, the percentage change is set to null, avoiding invalid calculations.
* The final SELECT statement:Retrieves the top 5 jurisdictions with the highest percentage change in weekly COVID death rates, excluding null values.Orders the results by Pct_change_weekly in descending order to highlight the regions with the largest increases.


**Output:**

|Residence_jurisdiction|	Period|	Data_Period_start|	Data_Period_end|	Total_Covid_Prct|	Prev_Week_Prct|	Pct_change_weekly|
|------|-------|--------|---------|---------|--------|------------|
|United States|	weekly|	2020-03-15|	2020-03-21|	1|	0.1|	900|
|Michigan|	weekly|	2020-03-22|	2020-03-28|	9.5|	1.1|	763.636363636364|
|Region 5|	weekly|	2020-03-22|	2020-03-28|	4|	0.6|	566.666666666667|
|Region 3|	weekly|	2020-03-22|	2020-03-28|	1.9|	0.3|	533.333333333333|
|New Jersey|	weekly|	2020-03-22|	2020-03-28|	13.7|	2.3|	495.652173913043|
|Region 1|	weekly	|2020-03-22|	2020-03-28|	4.1|	0.7|	485.714285714286|
|New York	|weekly|	2020-03-22|	2020-03-28|	11.4|	2|	470|
|Region 2|	weekly|	2020-03-22|	2020-03-28|	19.5|	3.6|	441.666666666667|
|Illinois|	weekly|	2020-03-22|	2020-03-28|	3.2|	0.6|	433.333333333333|
|Maryland|	weekly|	2020-03-29|	2020-04-04|	8.3|	1.6|	418.75|



**Question 7:What are the cumulative COVID deaths for each jurisdiction up to the latest data period end date?**

In [ ]:
SELECT 
   Residence_Jurisdiction, 
    SUM(COVID_deaths) AS cumulative_COVID_deaths
FROM 
    CovidP
WHERE 
    data_period_end <= (SELECT MAX(data_period_end) FROM CovidP)
    AND Period = 'Weekly'
GROUP BY 
    Residence_Jurisdiction
ORDER BY 
    cumulative_COVID_deaths DESC;

**Query Logic:**
* The subquery (SELECT MAX(data_period_end) FROM CovidP) finds the latest available data period end date in the dataset.
* The WHERE clause ensures that only records up to and including this latest data period are considered, filtering for weekly data (Period = 'Weekly').
* The SUM() function is used to calculate the cumulative number of COVID-19 deaths for each jurisdiction.
* The GROUP BY clause groups the data by each Residence_Jurisdiction, ensuring that the cumulative deaths are calculated for each region.
* Finally, the results are sorted in descending order of cumulative COVID-19 deaths using ORDER BY cumulative_COVID_deaths DESC, so the jurisdictions with the highest death toll appear at the top.

**Output:**

|Residence_Jurisdiction|	cumulative_COVID_deaths|
|--------|---------|
|United States|	1122637|
|Region 4|	252222|
|Region 5|	183647|
|Region 6|	159300|
|Region 9|	149682|
|Region 2|	121156|
|Region 3|	106851|
|California|	106826|
|Texas|	101145|
|Florida|	78144|
|Pennsylvania|	52003|
|Region 7|	49186|
|Ohio|	48974|
|Region 1|	44079|
|New York|	41154|
|Illinois|	39876|
|New York City|	39069|
|Michigan|	37118|
|Georgia|	35635|
|New Jersey|	34970|
|North Carolina|	33200|
|Region 8|	31857|
|Region 10|	30522|
|Arizona|	29248|
|Tennessee|	28126|
|Indiana|	25575|
|Virginia|	23213|
|Missouri|	22157|
|Alabama|	22047|
|South Carolina|	20942|
|Massachusetts|	20620|
|Kentucky|	19510|
|Oklahoma|	19226|
|Maryland|	17836||
|Louisiana|	17394|
|Wisconsin|	16857|
|Minnesota|	15231|
|Colorado|	14689|
|Washington|	14603|
|Mississippi|	14526|
|Arkansas|	12327|
|Connecticut|	12071|
|Nevada|	11685|
|Iowa|	10909|
|Kansas|	10383|
|New Mexico|	9073|
|Oregon|	8973|
|West Virginia|	8444|
|Puerto Rico|	5797|
|Nebraska|	5413|
|Idaho|	5230|
|Utah|	5228|
|Rhode Island|	3533|
Montana	|3433|
|Delaware|	3023|
|Maine|	2931||
|New Hampshire|	2899|
|South Dakota|	2804|
|North Dakota|	2134|
|Wyoming|	1711|
|Hawaii|	1400|
|District of Columbia|	1394|
|Alaska|	1079|
|Vermont|467|

**Question 8:Which Top 10 jurisdiction experienced the highest percentage increase in COVID deaths from the previous week, and what were the actual death numbers for each week?**

In [ ]:
WITH Previous_Week_Deaths AS (
    SELECT 
        Residence_Jurisdiction,
        Period,
        Covid_Deaths,
        LAG(Covid_Deaths, 1) OVER (PARTITION BY Residence_Jurisdiction ORDER BY Period) AS Previous_Week_Deaths
    FROM covidp
    WHERE Period = 'Weekly'
)
SELECT Top 10
    Residence_Jurisdiction,
    Period,
    Covid_Deaths,
    Previous_Week_Deaths,
    CASE 
        WHEN Previous_Week_Deaths = 0 THEN NULL
        ELSE ((Covid_Deaths - Previous_Week_Deaths) / Previous_Week_Deaths) * 100
    END AS Pct_Increase_Weekly
FROM Previous_Week_Deaths
WHERE Period = 'Weekly'
ORDER BY Pct_Increase_Weekly DESC;

**Query Logic:**

* The subquery (Previous_Week_Deaths) uses the LAG() window function to calculate the number of COVID deaths for the previous week for each jurisdiction. It partitions the data by jurisdiction and orders it by the Period to calculate the lag (previous week's value).
* In the main query, the results are filtered to include only the 'Weekly' periods, and the percentage increase is calculated for each jurisdiction.
* The calculation of percentage increase is done using the formula:((Covid_Deaths - Previous_Week_Deaths) / Previous_Week_Deaths) * 100 This gives the week-over-week percentage change in deaths.
* If the number of deaths in the previous week was zero, the percentage increase is set to NULL to avoid division by zero.
* The query then returns the top 10 jurisdictions with the highest percentage increase in weekly COVID deaths, ordering the results in descending order of percentage increase.

**Output:**

|Residence_Jurisdiction|	Period|	Covid_Deaths|	Previous_Week_Deaths|	Pct_Increase_Weekly|
|------|------|-------|--------|--------|
|Michigan|	weekly|	235|	22|	900|
|United States	|weekly|	586|	60|	800|
|New Jersey|	weekly|	293|	38|	600|
|New York City|	weekly|	997|	126|	600|
|Region 2|	weekly|	1594|	211|	600|
|New York|	weekly|	294|	44|	500|
|Region 1|	weekly|	119|	19|	500|
|Region 3|	weekly|	118|	18|500|
|Region 5|	weekly|	440|	66|	500|
|Illinois|	weekly|	75|	14|	400|

## Conclusion

**1. Jurisdiction with Highest COVID-19 Deaths for the Latest Period:** The jurisdiction with the highest number of COVID-19 deaths for the latest data period end date is **United States**. This indicates a significant impact of the pandemic in this region, with the highest death toll compared to others.

**2. Week-over-Week Percentage Change in Crude COVID Rate:** The week-over-week percentage change in the crude COVID rate shows that **Michigan** experienced the highest rate of increase. The top 10 jurisdictions exhibit notable variations, with some regions showing a sharp rise in COVID-19 spread over time.

**3. Top 5 Jurisdictions with the Highest Percentage Difference Between aa_COVID_Rate and Crude COVID Rate:** The top 5 jurisdictions with the highest percentage difference between the age-adjusted COVID rate and the crude COVID rate reveal significant demographic disparities in the pandemic's impact. These differences suggest that age distribution played a major role in mortality rates across these regions.

**4. Average Weekly COVID Deaths per Jurisdiction (Last Four Periods):** The average number of weekly COVID deaths across jurisdictions shows a consistent trend in certain regions, with **United States** having the highest weekly death toll. This highlights persistent COVID-19 mortality in certain areas over the last four data periods.

**5. Jurisdictions with Non-Zero COVID Deaths and No Missing Data:** The jurisdictions that reported non-zero COVID deaths and had no missing data provide a more reliable and comprehensive view of the pandemic's effect. This ensures data accuracy for further analysis and comparison.

**6. Week-over-Week Change in Total COVID Death Percentage (Since March 2020):** The top 10 jurisdictions with the highest total COVID deaths show varied week-over-week percentage changes since March 1, 2020. Some regions have experienced more gradual increases, while others showed significant spikes during key pandemic waves.

**7. Cumulative COVID Deaths by Jurisdiction:** The cumulative COVID deaths up to the latest data period indicate that **United States** continues to lead in total deaths, reflecting the long-term impact of COVID-19 in this region. This highlights areas that were hit hardest throughout the pandemic.

**8. Top 10 Jurisdictions with Highest Percentage Increase in COVID Deaths (Previous Week):** **Michigan** had the highest percentage increase in COVID deaths from the previous week. This rise indicates an emerging hot spot in the pandemic, with rapid growth in mortality over a short period, demanding further attention to contain the spread.